# Select frequency-range for Fourier transform in CSEM modelling



## Note
- Depending on the signal the real or imaginary part is used:
  - `signal >= 0`: Sine transform, imaginary part is used,
  - `signal < 0`: Cosine transform, real part is used.
- The 1D-model is for an inline-survey:
  - `src = [0, 0, src_z]` (at the origin at a given depth),
  - `rec = [off, 0, rec_z]` (in-line receiver at a given offset and depth).

In [1]:
import emg3d
import empymod
import numpy as np
import matplotlib.pyplot as plt

### `frequency-selection`

The frequency-selection GUI has its own repo with more examples.
Check-out: https://github.com/empymod/frequency-selection
The relevant part for this publication is copied here into `freqselect.py` in this directory.

In [2]:
import freqselect  # Script in this directory

In [3]:
%matplotlib ipympl

In [4]:
#%%html
#<style>
#  .output_wrapper .ui-dialog-titlebar {display: none;}
#  .container { width:100% !important; }
#</style>

## Model

In [5]:
time = np.logspace(-2, 3, 301)  # Times; select a wide range.
src_z = 950                     # Source depth.
rec_z = 1000                    # Receiver depth.
depth = [0, 1000, 2000, 2100]   # Interfaces (m).
res = [2e14, 0.3, 1, 100, 1]    # Resistivities (Ohm.m).

rec = [np.array([1.5, 3, 6, 12])*1e3, np.zeros(4), rec_z]

# Collect it in a dict.
model = {
    'src': [0, 0, src_z],
    'rec': rec,
    'depth': depth,
    'res': res,
}

## Interactive frequency-selection

In [6]:
GUI1 = freqselect.InteractiveFrequency(
    src_z=src_z,
    rec_z=rec_z,
    depth=depth,
    res=res,
    time=time,
    ft='fftlog',
    xtfact=3,
    linlog='log',
)
#plt.savefig('../figures/GUI-FFTLog-log.pdf', bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
GUI2 = freqselect.InteractiveFrequency(
    src_z=src_z,
    rec_z=rec_z,
    depth=depth,
    res=res,
    time=time,
    ft='dlf',
    ftarg={'dlf': 'key_81_CosSin_2009'},
    xtfact=3,
    linlog='linear',
)
#plt.savefig('../figures/GUI-DLF-lin.pdf', bbox_inches='tight')

In [8]:
GUI = freqselect.InteractiveFrequency(
    src_z=src_z,
    rec_z=rec_z,
    depth=depth,
    res=res,
    time=time,
    ft='fftlog',
    xtfact=3,
)

## Create a `Fourier` instance using the above parameters

In [9]:
Fourier = emg3d.utils.Fourier(
    time=GUI.time,      # Current times from the GUI.
    fmin=GUI.fmin,      # Current fmin from the GUI.
    fmax=GUI.fmax,      # Current fmax from the GUI.
    signal=GUI.signal,  # Current signal from the GUI.
    ft=GUI.ft,          # Current Fourier transform from the GUI.
    ftarg=GUI.ftarg,    # Current FT arguments from the GUI.
)

   time        [s] :  0.01 - 1000 : 301  [min-max; #]
   Fourier         :  FFTLog
     > pts_per_dec :  5
     > add_dec     :  [-2.  1.]
     > q           :  0.0
   Req. freq  [Hz] :  2.00364E-06 - 126.421 : 40  [min-max; #]
   Calc. freq [Hz] :  0.00126421 - 7.97664 : 20  [min-max; #]


### Calculate $f-$ and $t-$domain responses

In [10]:
freq_dense = np.logspace(np.log10(Fourier.freq_req.min()/2), np.log10(Fourier.freq_req.max()*2), 301)
fdata_dense = empymod.dipole(freqtime=freq_dense, **model)
fdata_comp = empymod.dipole(freqtime=Fourier.freq_req, **model)
fdata_calc = empymod.dipole(freqtime=Fourier.freq_calc, **model)                    # What we could model in 3D.
tdata_comp = empymod.dipole(freqtime=Fourier.time, signal=Fourier.signal, **model)  # For comparison reasons.


:: empymod END; runtime = 0:00:00.328559 :: 1 kernel call(s)


:: empymod END; runtime = 0:00:00.049715 :: 1 kernel call(s)


:: empymod END; runtime = 0:00:00.034113 :: 1 kernel call(s)


:: empymod END; runtime = 0:00:00.327273 :: 1 kernel call(s)



### Fourier transform

**Note:** The `fdata_req`-step is not necessary, that happens internally when calling `freq2time`. We just do it so we can plot the result.

In [11]:
# Pre-allocate arrays.
fdata_req = np.zeros((Fourier.freq_req.size, rec[0].size), dtype=complex)
tdata = np.zeros((Fourier.time.size, rec[0].size))

# Loop over offsets.
for i, off in enumerate(rec[0]):
    fdata_req[:, i] = Fourier.interpolate(fdata_calc[:, i])
    tdata[:, i] = Fourier.freq2time(fdata_calc[:, i], off)
    
fd_err = 100*abs((fdata_req-fdata_comp)/fdata_comp)
td_err = 100*abs((tdata-tdata_comp)/tdata_comp)

## Plot it

In [12]:
ls = ['-', '--', '-.', ':']
mk = ['o', '*', 'v', 'd']
ms = [5, 6, 6, 5]

plt.figure(figsize=(9, 5))

# f-domain
ax1 = plt.subplot2grid((6, 2), (0, 0), rowspan=4)
plt.title('(a) frequency domain')

for i, off in enumerate(rec[0]):
    norm = max(abs(fdata_req[:, i].imag))  # Normalize by max
    plt.plot(freq_dense, fdata_dense[:, i].imag/norm, f'.6', lw=2)
    plt.plot(Fourier.freq_req[~Fourier.freq_calc_i], fdata_req[~Fourier.freq_calc_i, i].imag/norm, 'k.')
    plt.plot(Fourier.freq_calc, fdata_calc[:, i].imag/norm, f'C{i}{mk[i]}', ms=ms[i])
    
    plt.plot(-1, fdata_calc[0, i].imag, f'C{i}{ls[i]}{mk[i]}', ms=ms[i], label=off/1e3)

plt.ylabel('Normalized field ($\Im(E)\ /\ \mathrm{max}|\Im(E)|$)')
plt.xscale('log')
plt.legend(title='Offset (km)', loc=4)
plt.grid(axis='y', c='0.9')
ax1.set_xticklabels([])

# f-domain error
ax2 = plt.subplot2grid((6, 2), (4, 0), rowspan=2)

for i in range(fd_err.shape[1]):
    plt.plot(Fourier.freq_req, fd_err[:, i], ls[i], lw=2)
# plt.plot(Fourier.freq_req[~Fourier.freq_calc_i], err[~Fourier.freq_calc_i, ii], 'k.')
# plt.plot(Fourier.freq_calc, err[Fourier.freq_calc_i, ii], 'C0.')
plt.axhline(1, color='.4', zorder=1)

plt.xscale('log')
plt.yscale('log')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Rel. error %')
plt.xlim(ax1.get_xlim())
plt.ylim([8e-2, 120])
plt.yticks([0.01, 0.1, 1, 10, 100], ('0.01', '0.1', '1', '10', '100'))
plt.grid(axis='y', c='0.9')

# t-domain
ax3 = plt.subplot2grid((6, 2), (0, 1), rowspan=4)
plt.title('(b) time domain')

for i, off in enumerate(rec[0]):
    norm = max(tdata[:, i])
    plt.plot(time, tdata_comp[:, i]/norm, '.6', lw=2)
    plt.plot(time, tdata[:, i]/norm, ls[i], lw=2, label=off/1e3)

plt.ylabel('Normalized field ($E\ / \ \mathrm{max}|E|$)')
plt.xscale('log')
plt.yscale('log')
plt.xlim([.05, 300])
plt.ylim([1e-3, 2e0])
# plt.legend(title='Offset (km)', ncol=2)
plt.grid(axis='y', c='0.9')

ax3.yaxis.set_ticks_position('right')
ax3.yaxis.set_label_position('right')
ax3.set_xticklabels([])

# t-domain error
ax4 = plt.subplot2grid((6, 2), (4, 1), rowspan=2)

for i in range(td_err.shape[1]):
    plt.plot(time, td_err[:, i], ls[i])
plt.axhline(1, color='.4', zorder=1)

plt.xscale('log')
plt.yscale('log')
plt.xlabel('Time (s)')
plt.ylabel('Rel. error %')
plt.xlim(ax3.get_xlim())
plt.ylim([8e-2, 120])
plt.yticks([0.01, 0.1, 1, 10, 100], ('0.01', '0.1', '1', '10', '100'))
ax4.yaxis.tick_right()
ax4.yaxis.set_label_position("right")
plt.grid(axis='y', c='0.9')

# Switch off spines
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax3.spines['top'].set_visible(False)
ax3.spines['left'].set_visible(False)
ax4.spines['top'].set_visible(False)
ax4.spines['left'].set_visible(False)

plt.tight_layout()
#plt.savefig('../figures/multi-offset.pdf', bbox_inches='tight')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
emg3d.Report(empymod)

--------------------------------------------------------------------------------
  Date: Thu Oct 15 19:29:26 2020 CEST

                OS : Linux
            CPU(s) : 4
           Machine : x86_64
      Architecture : 64bit
               RAM : 15.5 GB
       Environment : Jupyter

  Python 3.8.6 | packaged by conda-forge | (default, Oct  7 2020, 19:08:05)
  [GCC 7.5.0]

           empymod : 2.0.3
             numpy : 1.19.2
             scipy : 1.5.2
             numba : 0.50.1
             emg3d : 0.13.1.dev4+g907839c
            xarray : 0.16.1
        discretize : 0.5.1
              h5py : 2.10.0
        matplotlib : 3.3.2
              tqdm : 4.47.0
           IPython : 7.16.1

  Intel(R) Math Kernel Library Version 2020.0.1 Product Build 20200208 for
  Intel(R) 64 architecture applications
--------------------------------------------------------------------------------